In [ ]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)

In [ ]:
#import population data
pop_all_data = pd.read_csv('../cleaned_data/pop_all_data.csv')
pop_all_data

In [ ]:
#import tax return data
state_returns = pd.read_csv('../cleaned_data/state_returns.csv')
state_returns

In [ ]:
#Import state finance data
state_finances2 = pd.read_csv('../cleaned_data/state_finances2.csv')
state_finances2

In [ ]:
#first join
first_join = pd.merge(pop_all_data, state_returns, on = ['Statefips','Countyfips'], how = 'inner')

In [ ]:
#final join
final_join = pd.merge(first_join, state_finances2, on = ['Statefips'], how = 'left')
final_join

In [ ]:
#creating a dictionary to map state fips name
state_fips_dict = {0.0: 'United States', 1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas', 6: 'California', 8: 'Colorado',
                   9: 'Connecticut', 10: 'Delaware', 11: 'District of Columbia', 12: 'Florida', 13: 'Georgia', 15: 'Hawaii', 16: 'Idaho',
                   17: 'Illinois', 18: 'Indiana', 19: 'Iowa', 20: 'Kansas', 21: 'Kentucky', 22: 'Louisiana', 23: 'Maine', 24: 'Maryland',
                   25: 'Massachusetts', 26: 'Michigan', 27: 'Minnesota', 28: 'Mississippi', 29: 'Missouri', 30: 'Montana', 31: 'Nebraska',
                   32: 'Nevada', 33: 'New Hampshire', 34: 'New Jersey', 35: 'New Mexico', 36: 'New York', 37: 'North Carolina', 38: 'North Dakota',
                   39: 'Ohio', 40: 'Oklahoma', 41: 'Oregon', 42: 'Pennsylvania', 44: 'Rhode Island', 45: 'South Carolina', 46: 'South Dakota',
                   47: 'Tennessee', 48: 'Texas', 49: 'Utah', 50: 'Vermont', 51: 'Virginia', 53: 'Washington', 54: 'West Virginia', 55: 'Wisconsin',
                   56: 'Wyoming'}

In [ ]:
#mapping state name using dictionary and state fips. this will create a sepearte column with only the state name.
final_join['State_Name'] = final_join['Statefips'].map(state_fips_dict)

In [ ]:
final_join.head(4)

In [ ]:
final_join = final_join.drop(columns=['Location_x','Net International Migration','Net Domestic Migration','State','Taxable interest','Ordinary dividends',
                                      'Qualified dividends [8]','Business or profession\nnet income (less loss)','Net capital gain (less loss) in AGI','Partnership/S-corp net income (less loss)',
                                      'Partnership/S-corp net income (less loss)','Total statuory adjustments','Educator expenses','Self-employed (Keogh) retirement plans',
                                      'Self-employed health insurance deduction','Individual retirement arrangement payments','Student loan interest deduction','Basic standard deduction',
                                      'Additional standard deduction','Total itemized deductions','Total medical and dental \nexpenses deduction','Limited state and local taxes paid','Home mortgage interest paid ',
                                      'Home mortgage interest paid \nto personal seller','Deductible points','Investment interest paid','Total charitable contributions','Other non-limited miscellaneous deduction',
                                      'Qualified business income deduction\n','Income tax before credits','Alternative minimum tax','Excess advance \npremium tax credit repayment','Total tax credits [10]',
                                      'Foreign tax credit','Child and dependent care credit','Nonrefundable education credit','Retirement savings contributions credit','Child and other dependent credit',
                                      'Residential energy tax credit','Self-employment tax','Total premium tax credit','Advance premium tax credit','Total tax payments','Earned income credit [11]',
                                      'Excess earned income credit \n(refundable) [12]','Additional child tax credit','Refundable education credit [13]','Net premium tax credit','Income tax after credits',
                                      'Total additional Medicare tax','Net investment income tax','Tax due at time of filing [15]','Total overpayments','Overpayments refunded [16]',
                                      "Credited to next year's estimated tax",'Location','Debt Outstanding', 'Short-Term Debt', 'Long-Term Debt', 'Public Debt For Private Purposes', 'Long-Term Debt Issued',
                                      'Long-Term Debt Retired','Cash And Security Holdings', 'Insurance Trust Funds','Unemployment Compensation','Employee Retirement',	"Workers' Compensation", 'Miscellaneous',
                                      'Other Than Insurance Trust Funds','Offsets To Debt','Bond Funds','Other','Other Insurance Trust Expenditure'])

In [ ]:
column_to_drop = 'Home mortgage interest paid'

# Check if the column exists before attempting to drop it
if column_to_drop in final_join.columns:
    final_join = final_join.drop(columns=[column_to_drop], errors='ignore')
    print(f"Column '{column_to_drop}' successfully dropped.")
else:
    print(f"Column '{column_to_drop}' not found in DataFrame.")

In [ ]:
final_join.columns = final_join.columns.str.strip()

In [ ]:
final_join.head(3)


In [ ]:
#rearranging columns for ease of use
final_join = final_join[['State_Name', 'Statefips', 'Location_y', 'Countyfips'] +
                        [col for col in final_join.columns if col not in ['State_Name', 'Location_y', 'Statefips', 'Countyfips']]]

In [ ]:
#rename columns
final_join = final_join.rename(columns={'State_Name': 'State', 'Location_y': 'County'})


In [ ]:
final_join['County'].iloc[0] = np.nan

In [ ]:
final_join

Creating multiple new columns for final dashboarding to include correlations

In [ ]:
final_join['Average Taxable Income'] = (final_join['Taxable income'] / final_join['Total Population'])

In [ ]:
final_join['Average Taxes Paid'] = (final_join['Total tax liability [14]'] / final_join['Total Population'])

In [ ]:
#not  divided by total population as this will provide data for entire region
final_join['Percent Of Taxable Income Paid In Taxes'] = (final_join['Total tax liability [14]'] / final_join['Taxable income'])

Creating correlations to be used on final dashboard

In [ ]:
final_join['Average Taxable Income Corr'] = final_join['Life Expectancy'].corr(final_join['Average Taxable Income'])

In [ ]:
final_join['Average Wage Corr'] = final_join['Life Expectancy'].corr(final_join['Average Wage'])

In [ ]:
final_join['Average State and Local Income Taxes Paid Corr'] = final_join['Life Expectancy'].corr(final_join['State and local income taxes'] / final_join['Total Population'])

In [ ]:
final_join['Average State and Local General Sales Taxes Paid Corr'] = final_join['Life Expectancy'].corr(final_join['State and local general sales taxes'] / final_join['Total Population'])


In [ ]:
final_join['Average Taxes Paid Corr'] = final_join['Life Expectancy'].corr(final_join['Average Taxes Paid'])

In [ ]:
final_join['Percent Of Taxable Income Paid in Taxes Corr'] = final_join['Life Expectancy'].corr(final_join['Percent Of Taxable Income Paid In Taxes'])


In [ ]:
final_join['State Tax Revenue Corr'] = final_join['Life Expectancy'].corr(final_join['Tax Revenue'])

In [ ]:
final_join['Government Services Revenue Corr'] = final_join['Life Expectancy'].corr(final_join['Total Charges And Miscellaneous General Revenue'])

In [ ]:
final_join['Federal Tax Revenue Corr'] = final_join['Life Expectancy'].corr(final_join['From Federal Government Revenue'])

In [ ]:
final_join['Insurance Trust Revenue Corr'] = final_join['Life Expectancy'].corr(final_join['Total Insurance Trust Revenue'])

In [ ]:
final_join['Utility Revenue Corr'] = final_join['Life Expectancy'].corr(final_join['Total Utility Revenue'])

In [ ]:
final_join['Liquor Store Revenue Corr'] = final_join['Life Expectancy'].corr(final_join['Total Liquor Store Revenue'])

In [ ]:
final_join['Social Services And Income Maintenance Expenditure Corr'] = final_join['Life Expectancy'].corr(final_join['Total Social Services And Income Maintenance Expenditure:'])

In [ ]:
final_join['Education Services Expenditure Corr'] = final_join['Life Expectancy'].corr(final_join['Total Education Services Expenditure:'])

In [ ]:
final_join['Insurance Trust Expenditure Corr'] = final_join['Life Expectancy'].corr(final_join['Total Insurance Trust Expenditure:'])

In [ ]:
final_join['Public Safety Expenditure Corr'] = final_join['Life Expectancy'].corr(final_join['Total Public Safety Expenditure:'])

In [ ]:
final_join['Utility Expenditure Corr'] = final_join['Life Expectancy'].corr(final_join['Total Utility Expenditure:'])

In [ ]:
final_join['Transportation Expenditure Corr'] = final_join['Life Expectancy'].corr(final_join['Total Transportation Expenditure:'])

In [ ]:
final_join['Governmental Administration Expenditure Corr'] = final_join['Life Expectancy'].corr(final_join['Total Governmental Administration Expenditure:'])

In [ ]:
final_join['Environment and Housing Expenditure Corr'] = final_join['Life Expectancy'].corr(final_join['Total Environment And Housing Expenditure:'])

CODE USED TO DETERMINE IF ANY SIGNIFICANT CORRELATION EXISTED BETWEEN LIFE EXPECTANCY AND ANY SUB CATAGORY OF REVENUE OR EXPENDITURE.
SINCE NO SIGNIFICANT CORRELATION EXISTED, CODE IS NOT USED FOR FINAL DASHBOARD BUT IS AVAIABLE

target_column = 'Life Expectancy'

#Find the starting index of "Revenue"
start_index = final_join.columns.get_loc("Revenue")

#List of columns to calculate correlations with
other_columns = final_join.columns[start_index+1:]

#Calculate correlations and create new columns with correlation values
for col in other_columns:
    correlation_value = final_join[target_column].corr(final_join[col])
    new_col_name = f"Corr {col}"
    final_join[new_col_name] = correlation_value

In [ ]:
final_join.head(3)

In [ ]:
master_table = final_join.copy()
master_table.to_csv('../cleaned_data/master_table.csv',index=False)

In [ ]:
master_table[master_table['Countyfips']==0].head(2)

In [ ]:
final_join['Average Wage Corr'].mean